# Model Registry: Train, Register, and Serve R Models

This notebook demonstrates the `snowflakeR` package workflow for registering R models
in the Snowflake Model Registry and running inference.

**What you'll do:**
1. Train models in R (as usual)
2. Test predictions locally
3. Register to Snowflake Model Registry with one function call
4. Manage versions and metrics
5. Run remote inference via SPCS

**Under the hood:** `snowflakeR` auto-generates a Python `CustomModel` wrapper
that uses `rpy2` to load and call your R model. You never write Python.

This notebook is for **local R environments** (RStudio, Posit Workbench, JupyterLab with R kernel).
For Snowflake Workspace Notebooks, use `workspace_model_registry.ipynb`.

**Before you start:** Copy `notebook_config.yaml.template` to `notebook_config.yaml`
and edit it with your account, warehouse, database, and schema.

**Sections:**
1. Setup
2. Connect
3. Train a Model
4. Test Locally
5. Register to Snowflake
6. Manage Models
7. Remote Inference
8. Advanced: Custom Predict Code
9. Cleanup

---

## 1. Setup

Install snowflakeR and set up the Python environment (one time only).

```r
# install.packages("pak")
# pak::pak("Snowflake-Labs/snowflakeR")
# snowflakeR::sfr_install_python_deps()
```

In [ ]:
library(snowflakeR)

---
## 2. Connect & Set Execution Context

`sfr_load_notebook_config()` reads `notebook_config.yaml` and runs
`USE WAREHOUSE / DATABASE / SCHEMA` to set the execution context.

All table references use fully qualified names via `sfr_fqn()` for
consistency with Workspace Notebooks.

In [ ]:
# Connect (reads ~/.snowflake/connections.toml or config connection params)
conn <- sfr_connect()

# Load config and set execution context
conn <- sfr_load_notebook_config(conn)
conn

# Create a Model Registry context
reg <- sfr_model_registry(conn)
reg

---
## 3. Train a Model

### Example A: Linear regression on mtcars

Train any R model as you normally would -- nothing snowflakeR-specific here.

In [ ]:
# Train a simple linear model
model_lm <- lm(mpg ~ wt + hp + cyl, data = mtcars)
summary(model_lm)

### Example B: Time series with forecast

```r
library(forecast)
model_arima <- auto.arima(AirPassengers)
summary(model_arima)
```

---

## 4. Test Locally

`sfr_predict_local()` runs the **exact same prediction logic** that will execute
inside Snowflake, but entirely in R. Use this to verify before registering.

In [ ]:
# Create test data
test_data <- data.frame(
  wt  = c(2.5, 3.0, 3.5, 4.0),
  hp  = c(110, 150, 200, 245),
  cyl = c(4L, 6L, 8L, 8L)
)

# Test locally -- same predict path as remote
preds <- sfr_predict_local(model_lm, test_data)
cbind(test_data, preds)

---
## 5. Register to Snowflake

One call to `sfr_log_model()` handles everything:
- Saves the R model to `.rds`
- Auto-generates a Python `CustomModel` wrapper
- Registers in the Snowflake Model Registry

In [ ]:
mv <- sfr_log_model(
  reg,
  model        = model_lm,
  model_name   = "SFR_DEMO_MPG",
  version_name = "V1",
  input_cols   = list(wt = "double", hp = "double", cyl = "integer"),
  output_cols  = list(prediction = "double"),
  comment      = "Linear regression: MPG from weight, horsepower, cylinders"
)

mv

### Key parameters for `sfr_log_model()`

| Parameter | Description |
|-----------|-------------|
| `model` | Any R object that can be `saveRDS()`'d |
| `model_name` | Registry name (uppercase recommended) |
| `input_cols` | Named list: column name -> type (`double`, `integer`, `string`, `boolean`) |
| `output_cols` | Named list: output column name -> type |
| `predict_fn` | R function name (default: `"predict"`) |
| `predict_pkgs` | R packages needed at inference time |
| `conda_deps` | Extra conda packages (r-base and rpy2 always included) |
| `target_platforms` | `"SNOWPARK_CONTAINER_SERVICES"` (default) or `"WAREHOUSE"` |

---

## 6. Manage Models

### List and inspect models

In [ ]:
# List all models in the registry
models <- sfr_show_models(reg)
models

In [ ]:
# Get a specific model
m <- sfr_get_model(reg, "SFR_DEMO_MPG")
m

# Show versions
sfr_show_model_versions(reg, "SFR_DEMO_MPG")

### Metrics

Attach evaluation metrics to model versions:

In [ ]:
# Calculate and set metrics
preds_train <- predict(model_lm, mtcars)
rmse <- sqrt(mean((mtcars$mpg - preds_train)^2))
r_sq <- summary(model_lm)$r.squared

sfr_set_model_metric(reg, "SFR_DEMO_MPG", "V1", "rmse", rmse)
sfr_set_model_metric(reg, "SFR_DEMO_MPG", "V1", "r_squared", r_sq)

cat(sprintf("RMSE: %.3f, R-squared: %.3f\n", rmse, r_sq))

In [ ]:
# Retrieve metrics
sfr_show_model_metrics(reg, "SFR_DEMO_MPG", "V1")

### Log a second version

In [ ]:
# Train a better model (added displacement)
model_v2 <- lm(mpg ~ wt + hp + cyl + disp, data = mtcars)

mv2 <- sfr_log_model(
  reg,
  model        = model_v2,
  model_name   = "SFR_DEMO_MPG",
  version_name = "V2",
  input_cols   = list(wt = "double", hp = "double", cyl = "integer", disp = "double"),
  output_cols  = list(prediction = "double"),
  comment      = "V2: added displacement"
)

# Set as default
sfr_set_default_model_version(reg, "SFR_DEMO_MPG", "V2")

---
## 7. Inference

### Local prediction (works everywhere)

`sfr_predict_local()` runs the **exact same prediction logic** that the registered model
uses, but entirely in your local R session. Use this to verify predictions before
deploying to production.

> **Note:** R models require `rpy2` and `r-base` at inference time, which are
> not available in the Snowflake warehouse Anaconda channel. Therefore, warehouse
> inference (`sfr_predict()` without a service) is not supported for R models.
> For production inference in Snowflake, deploy via SPCS (see below).

In [ ]:
# Test data
new_data <- data.frame(
  wt   = c(2.62, 3.44, 3.57),
  hp   = c(110, 175, 245),
  cyl  = c(4L, 6L, 8L),
  disp = c(120.3, 258.0, 360.0)
)

# Predict locally -- same logic as the registered model
preds <- sfr_predict_local(model_lm, new_data)
cbind(new_data, preds)

### Production inference via SPCS

For production inference **inside Snowflake**, deploy the model as a
Snowpark Container Services (SPCS) service. This creates a container with
R, rpy2, and your model, then serves predictions via a REST endpoint.

The cells below create the SPCS infrastructure (compute pool + image repo)
if it doesn't already exist, then deploy, wait, test, benchmark, and undeploy.
Each step is a separate cell so you can re-run individually.

In [ ]:
# ── 0. Create SPCS infrastructure (if not exists) ────────────────────────
sfr_create_compute_pool(conn, "R_FORECAST_POOL", instance_family = "CPU_X64_M")
sfr_create_image_repo(conn, sfr_fqn(conn, "R_FORECAST_IMAGES"))

# ── 1. Deploy as SPCS service (force = TRUE drops existing service first)
sfr_deploy_model(
  reg,
  model_name   = "SFR_DEMO_MPG",
  version_name = "V2",
  service_name = "mpg_service",
  compute_pool = "R_FORECAST_POOL",
  image_repo   = sfr_fqn(conn, "R_FORECAST_IMAGES"),
  force        = TRUE
)

In [ ]:
# ── 2. Wait for service to be ready ──────────────────────────────────────
# Polls every 15 seconds, times out after 10 minutes
sfr_wait_for_service(reg, "mpg_service", timeout_min = 10, poll_sec = 15)

In [ ]:
# ── 3. Run inference via the service ─────────────────────────────────────
preds <- sfr_predict(reg, "SFR_DEMO_MPG", new_data, service_name = "mpg_service")
preds

# ── 4. Benchmark: run N iterations and report latency stats ──────────────
bench <- sfr_benchmark_inference(
  reg, "SFR_DEMO_MPG", new_data,
  service_name = "mpg_service",
  n = 20                                        # adjust iterations as needed
)

In [ ]:
# ── 5. Undeploy when done ────────────────────────────────────────────────
sfr_undeploy_model(reg, "SFR_DEMO_MPG", "V2", "mpg_service")

## 8. Advanced -- Custom Predict Code

For models that need special prediction logic (e.g., `forecast`, multi-step pipelines),
use the `predict_body` template.

### Template variables

| Variable | Description |
|----------|-------------|
| `{{MODEL}}` | The loaded R model object |
| `{{INPUT}}` | The input data.frame |
| `{{UID}}` | Unique ID for variable naming |
| `{{N}}` | Number of rows in input |

In [ ]:
# Example: forecast model with custom output
# (Uncomment and adapt for your use case)

# library(forecast)
# arima_model <- auto.arima(AirPassengers)
#
# mv_forecast <- sfr_log_model(
#   reg,
#   model        = arima_model,
#   model_name   = "SFR_DEMO_FORECAST",
#   predict_fn   = "forecast",
#   predict_pkgs = c("forecast"),
#   predict_body = '
#     pred_{{UID}} <- forecast({{MODEL}}, h = {{N}})
#     result_{{UID}} <- data.frame(
#       period = seq_len({{N}}),
#       point_forecast = as.numeric(pred_{{UID}}$mean),
#       lower_95 = as.numeric(pred_{{UID}}$lower[,2]),
#       upper_95 = as.numeric(pred_{{UID}}$upper[,2])
#     )
#   ',
#   input_cols  = list(period = "integer"),
#   output_cols = list(
#     period = "integer", point_forecast = "double",
#     lower_95 = "double", upper_95 = "double"
#   )
# )

---

## 9. Cleanup

In [ ]:
# Uncomment to clean up demo objects
# (commented out to avoid accidental deletion on Run All)
#
# sfr_delete_model(reg, "SFR_DEMO_MPG")
# sfr_execute(conn, paste("DROP TABLE IF EXISTS", sfr_fqn(conn, "SFR_DEMO_PREDICT_INPUT")))
# sfr_disconnect(conn)
# cat("Cleanup complete.\n")

---

## Supported model types

Any R model that can be serialised with `saveRDS()` works:

- `lm()`, `glm()` (base R)
- `randomForest::randomForest()`
- `xgboost::xgb.train()`
- `ranger::ranger()`
- `forecast::auto.arima()`, `forecast::ets()`
- `tidymodels` workflows
- Custom S3/S4 model objects

## Next steps

- **Feature Store:** See `local_feature_store.ipynb`
- **Full documentation:** `vignette("model-registry", package = "snowflakeR")`